In [ ]:
# Water quality monitoring points
import arcpy

# Set up environment settings
arcpy.env.workspace = "C:\\Users\\qingyuel\\Desktop\\644"  # Update with your workspace path
arcpy.env.overwriteOutput = True

# Define the coordinate system
sr = arcpy.SpatialReference(4326)  # WGS 1984

# Create a new geodatabase
gdb_name = "WaterQuality.gdb"
arcpy.CreateFileGDB_management(arcpy.env.workspace, gdb_name)

# Set up fields
fields = ["latitude", "longitude", "sediment"]

# Create a new feature class in the geodatabase
fc_name = "MonitoringPoints"
output_fc = arcpy.CreateFeatureclass_management(gdb_name, fc_name, "POINT", spatial_reference=sr)

# Add fields to the feature class
for field in fields:
    arcpy.AddField_management(output_fc, field, "DOUBLE")

# Read water quality table and insert data into the feature class
water_quality_table = "C:\\Users\\qingyuel\\Desktop\\644\\sediment.csv"  # Update with the path to your water quality table
with arcpy.da.SearchCursor(water_quality_table, ["latitude", "longitude", "ResultMeasureValue"]) as cursor:
    with arcpy.da.InsertCursor(output_fc, ["SHAPE@", "latitude", "longitude", "sediment"]) as icursor:
        for row in cursor:
            point = arcpy.Point(row[1], row[0])  # Longitude, Latitude
            pt_geom = arcpy.PointGeometry(point, sr)
            icursor.insertRow([pt_geom, row[0], row[1], row[2]])

print("Feature class created successfully.")

In [ ]:
# Update the symbology of the monitoring points layer
import arcpy
import arcpy.mp

# Get the current project
project = arcpy.mp.ArcGISProject("CURRENT")

# Access the map within the project
map = project.listMaps()[0] 

# Access the layer within the map
layer = map.listLayers("MonitoringPoints")[0]

# Access the symbology of the layer
sym = layer.symbology

# Set symbology to proportional symbols
sym.renderer.symbolPath = "esriCartographyMarkersMarker3"
sym.renderer.symbolSize = "Field"
sym.renderer.normalization = None 
sym.renderer.minSymbolSize = 8
sym.renderer.maxSymbolSize = 30

# Set the field for proportional symbols
sym.renderer.field = "sediment"

# Apply changes
layer.symbology = sym

print("Symbology updated successfully.")

In [ ]:
# Open the North American Lakes and Rivers Data, and filter to Michigan state based on the Cartographic Boundary Files.

import arcpy
import os

# Paths to data
Water_Bodies_shapefile = "C:\\Users\\roujia\\Desktop\\644\\USA_Detailed_Water_Bodies\\USA_Detailed_Water_Bodies.shp"
state_boundary_shapefile = "C:\\Users\\roujia\\Desktop\\644\\mi_state_boundary\\mi_state.shp"
output_workspace = "C:\\Users\\roujia\\Desktop\\644"

# Output path for the clipped water bodies
output_waterbodies_fc = os.path.join(output_workspace, "Michigan_waterbodies.shp")

# Clip water bodies to Michigan boundary
arcpy.Clip_analysis(Water_Bodies_shapefile, state_boundary_shapefile, output_waterbodies_fc)

print("Water bodies clipped to Michigan boundary have been saved to a new feature class.")

In [ ]:
# Update the attribute table of the landcover raster
import arcpy

# Path to the landcover raster file
raster_file = "C:\\Users\\roujia\\Desktop\\644\\landcover_MI\\NA NALCMS landcover _1.tif"

# Define the landcover code dictionary
landcover_code = {
    1: "Temperate or sub-polar needleleaf forest",
    5: "Temperate or sub-polar broadleaf deciduous forest",
    6: "Mixed forest",
    8: "Temperate or sub-polar shrubland",
    10: "Temperate or sub-polar grassland",
    14: "Wetland",
    15: "Cropland",
    16: "Barren land",
    17: "Urban and built-up",
    18: "Water"
}

# Add a new field for "landcover type" to the attribute table of the raster
arcpy.AddField_management(raster_file, "Landcover", "TEXT")

# Update the "landcover type" field in the attribute table of the raster
with arcpy.da.UpdateCursor(raster_file, ["Value", "Landcover"]) as cursor:
    for row in cursor:
        value = row[0]
        if value in landcover_code:
            row[1] = landcover_code[value]
            cursor.updateRow(row)

print("Attribute table of the raster has been updated with the new 'LandcoverType' field.")


# Path to the landcover raster file
raster_file = "C:\\Users\\roujia\\Desktop\\644\\landcover_MI\\NA NALCMS landcover _1.tif"

# Open a new map in ArcGIS Pro
aprx = arcpy.mp.ArcGISProject("CURRENT")
map = aprx.listMaps()[0]  # Assuming there's only one map in the project

# Add the raster to the map
map.addDataFromPath(raster_file)

print("Raster file has been added to the map.")